<a href="https://colab.research.google.com/github/Shamima-Ali/mHack/blob/master/mHack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [271]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [272]:
dataset = pd.read_csv('hourly_fitbit_sema_df_unprocessed.csv')

# remove these cols
df2=dataset.drop(["date"], axis = 1)
df2=df2.drop(["id"], axis = 1)
df2=df2.drop(["temperature"], axis = 1)
df2=df2.drop(["badgeType"], axis = 1)
df2=df2.drop(["distance"], axis = 1)
df2=df2.drop(["mindfulness_session"], axis = 1)
df2=df2.drop(["scl_avg"], axis = 1)
df2=df2.drop(["steps"], axis = 1)
df2=df2.drop(["minutes_in_default_zone_1"], axis = 1)
df2=df2.drop(["minutes_below_default_zone_1"], axis = 1)
df2=df2.drop(["TENSE/ANXIOUS"], axis = 1)
df2=df2.drop(["minutes_in_default_zone_2"], axis = 1)
df2=df2.drop(["minutes_in_default_zone_3"], axis = 1)
df2=df2.drop(["SAD"], axis = 1)
df2=df2.drop(["TIRED"], axis = 1)
df2=df2.drop(["ENTERTAINMENT"], axis = 1)
df2=df2.drop(["GYM"], axis = 1)
df2=df2.drop(["HOME"], axis = 1)
df2=df2.drop(["HOME_OFFICE"], axis = 1)
df2=df2.drop(["step_goal"], axis = 1)
df2=df2.drop(["min_goal"], axis = 1)
df2=df2.drop(["max_goal"], axis = 1)
df2=df2.drop(["step_goal_label"], axis = 1)
df2=df2.drop(["ALERT"], axis = 1)
df2=df2.drop(["HAPPY"], axis = 1)
df2=df2.drop(["NEUTRAL"], axis = 1)
df2=df2.drop(["RESTED/RELAXED"], axis = 1)
df2=df2.drop(["OTHER"], axis = 1)
df2=df2.drop(["OUTDOORS"], axis = 1)
df2=df2.drop(["TRANSIT"], axis = 1)
df2=df2.drop(["WORK/SCHOOL"], axis = 1)
df2=df2.drop(["Unnamed: 0"], axis = 1)

# drop string values
df2=df2.dropna()
df2 = df2.drop(df2[df2.bmi == '<19'].index)
df2 = df2.drop(df2[df2.bmi == '>=25'].index)
df2 = df2.drop(df2[df2.bmi == '>30'].index)
df2 = df2.drop(df2[df2.bmi == '>=30'].index)

# rearrange cols
df2=df2[['hour', 'activityType', 'bpm', 'gender', 'bmi',  'calories']]

thisdict = {
  1: 'Walk',
  2: 'Bike',
  3: 'Workout"',
  4: 'Sport',
  5: 'Aerobic Workout'
}
df2.loc[df2['activityType'] == "['Walk']", 'activityType'] = 1
df2.loc[df2['activityType'] == "['Bike']", 'activityType'] = 2
df2.loc[df2['activityType'] == "['Workout']", 'activityType'] = 3
df2.loc[df2['activityType'] == "['Sport']", 'activityType'] = 4
df2.loc[df2['activityType'] == "['Aerobic Workout']", 'activityType'] = 5

# remove data that is not in the dictionary
df2 = df2.drop(df2[(df2.activityType != 1) & (df2.activityType != 2) 
& (df2.activityType != 3) & (df2.activityType != 4) & (df2.activityType != 5)].index)



X = df2.iloc[:, :-1].values
y = df2.iloc[:, -1].values


# print(X)
# print(y)


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5,10,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


1. ['Walk']
2. ['Bike']
3. ['Workout']
4. ['Sport']
5. ['Aerobic Workout']

In [273]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
labelEncoder = OneHotEncoder()
ct = ColumnTransformer(transformers=[('encoder', labelEncoder, [3])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
# ## MALE = 0.0 and 0.1

In [275]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [276]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [277]:
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)
y_res = np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1)

# Absolute error = measured value (predicted) - actual value (from csv file).
# for r in y_res:
#   value = r[0]-r[1]
#   print(value)


In [278]:
import pickle

In [281]:
data = {"model": regressor, "labelEncoder": labelEncoder}
with open ('MulRegModel.pkl', 'wb') as file:
  pickle.dump(data, file)

In [284]:
## checking the model
with open ('MulRegModel.pkl', 'rb') as file:
  data = pickle.load(file)

reg_loaded = data["model"]
labelEncoder_loaded = data["labelEncoder"]
y_pred_load = reg_loaded.predict(X_test)
# print(y_pred_load)
# y_res = np.concatenate((y_pred_load.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1)
# print(y_res)

[[172.18 119.26]
 [327.14 300.11]
 [201.42 237.85]
 [133.41 103.08]
 [268.86 282.82]
 [264.92 268.71]
 [224.21 182.71]
 [313.58 298.2 ]
 [375.78 352.59]
 [251.13 214.31]
 [227.31 210.52]
 [227.8  130.61]
 [170.57 231.9 ]
 [113.27 161.63]
 [222.41 255.44]
 [228.05 233.22]
 [275.89  97.35]
 [284.43 245.01]
 [193.63 193.83]
 [246.78 260.29]
 [206.79 138.46]
 [239.87 177.5 ]
 [231.25 109.22]
 [294.49 271.94]
 [432.13 723.69]
 [207.13 238.91]
 [199.74  61.96]
 [255.47 303.84]
 [282.11 184.56]
 [282.67 401.27]
 [221.4  287.4 ]
 [248.93 224.55]
 [204.71 172.98]
 [184.32 229.52]
 [267.02 206.11]
 [166.24 195.36]
 [298.71 237.55]
 [216.9  116.59]
 [222.83 208.11]
 [372.67 433.55]
 [186.57 142.58]
 [272.47 312.2 ]
 [246.7  162.  ]
 [184.76 137.4 ]
 [258.51 250.87]
 [263.7  324.21]
 [262.09 265.68]
 [173.09 253.92]
 [292.98 214.61]
 [412.99 503.05]
 [190.47 264.74]
 [303.46 567.67]
 [265.47 248.23]
 [ 85.56 145.22]
 [264.58 204.28]
 [240.55 202.87]
 [129.57 210.16]
 [277.87 389.  ]
 [119.45 170.5